In [1]:
from HD_DataLoader import *
from HD_DataLoader import SplitNormalFaulty
import os
folder_path = os.path.join(os.getcwd(), 'HD_data') 

example1 = HD(data_path = folder_path)

database = example1.generate_database()
# test, train, val  = example1.get_NN_Loaders()

---------------------------------------------------------
Generating database for HD's


In [30]:
# logmel_test = VairableSensorsDataset(database)

import librosa

test = SplitNormalFaulty(database)

working, faulty = test.split_faulty_working_mic()

working = np.array(working[0])

# for t in np.arange(round(self.start_perc*time[-1]),round(self.stop_perc*time[-1]),self.stride_sec):
                
#                 if self.mic_bool=='True':
#                     i = round(t*self.mic_fs)
#                     data_mic = database[data]['Microphone']['Data'][:,0]             
#                     data_mic = data_mic[i:i+round(self.window_sec*self.mic_fs)]
#                 else:
#                     data_mic = []

def _get_melspectogram_features(data,
                n_mels=64,
                n_frames=1,
                n_fft=1024,
                hop_length=512,
                power=2.0):
    """
    convert file_name to a vector array.

    file_name : str
        target .wav file

    return : numpy.array( numpy.array( float ) )
        vector array
        * dataset.shape = (dataset_size, feature_vector_length)
    """
    # calculate the number of dimensions
    dims = n_mels * n_frames

    sr = 441

    mel_spectrogram = librosa.feature.melspectrogram(y=data,
                                                    sr=sr,
                                                    n_fft=n_fft,
                                                    hop_length=hop_length,
                                                    n_mels=n_mels,
                                                    power=power,
                                                    center=False)

    # convert melspectrogram to log mel energies
    log_mel_spectrogram = 20.0 / power * np.log10(np.maximum(mel_spectrogram, sys.float_info.epsilon))

    # calculate total vector size
    n_vectors = len(log_mel_spectrogram[0, :]) - n_frames + 1

    # skip too short clips
    if n_vectors < 1:
        return np.empty((0, dims))

    # generate feature vectors by concatenating multiframes
    vectors = np.zeros((n_vectors, dims))
    for t in range(n_frames):
        vectors[:, n_mels * t : n_mels * (t + 1)] = log_mel_spectrogram[:, t : t + n_vectors].T

    return vectors

window_sec = 1
mic_fs = 44100

vector_test = _get_melspectogram_features(working, n_fft = mic_fs, hop_length=(window_sec*mic_fs))

# plt.plot(vector_test[6])
# plt.show()

In [33]:
# len(working)

44100*60

2646000